# This is a working file to create the functions to get current bus poistions from OBA api for Sound Transit agencies.
---

### Get OBA Info, Set Agency ID

In [73]:
with open('OBA_API_Key.txt') as key_file:
    key_mpm = key_file.read()
print "Key:", key_mpm

agency_ID = "1"
print "Agency ID:",agency_ID

Key: 66fae88a-fcc9-4ceb-958d-3932f378c161
Agency ID: 1


In [74]:
with open('OBA_Server_Location.txt', 'r') as server_file:
    oba_url = server_file.read()
print oba_url

http://api.pugetsound.onebusaway.org/


### Imports

In [75]:
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as BS
import csv

### Get 'Alerts' Trial

In [50]:
request_alerts = oba_url + 'api/gtfs_realtime/alerts-for-agency/'+agency_ID+'.pbtext?key=' + key_mpm + "&removeAgencyIds=false"
request_positions
request_alerts

'http://api.pugetsound.onebusaway.org/api/gtfs_realtime/alerts-for-agency/1.pbtext?key=66fae88a-fcc9-4ceb-958d-3932f378c161'

In [6]:
r = requests.get(request_alerts)
r.status_code

200

In [59]:
print r.content

header {
  gtfs_realtime_version: "1.0"
  timestamp: 1459837045
}



### Get 'Positions' Trial

In [51]:
request_positions = oba_url + 'api/gtfs_realtime/vehicle-positions-for-agency/' + agency_ID + '.pbtext?key=' + key_mpm + "&removeAgencyIds=false"
request_positions

'http://api.pugetsound.onebusaway.org/api/gtfs_realtime/vehicle-positions-for-agency/1.pbtext?key=66fae88a-fcc9-4ceb-958d-3932f378c161&removeAgencyIds=false'

In [52]:
r = requests.get(request_positions)
r.status_code

200

In [53]:
positions_raw = r.content

In [115]:
begin_header = positions_raw.find("header")
end_header = positions_raw.find("}")
pos_header = positions_raw[begin_header:end_header+1]

loc_entity = [i.start() for i in re.finditer("entity", positions_raw)]
pos_entities = []
for i in loc_entity:
    if i!=loc_entity[-1]:
        pos_entities.append(positions_raw[i:(loc_entity[loc_entity.index(i)+1]-1)])
    else:
        pos_entities.append(positions_raw[i:-1])


In [116]:
print pos_header
print pos_entities[7]

header {
  gtfs_realtime_version: "1.0"
  timestamp: 1459836597
}
entity {
  id: "8"
  vehicle {
    trip {
      trip_id: "1_30973639"
      route_id: "1_100225"
    }
    position {
      latitude: 47.690582
      longitude: -122.35099
    }
    timestamp: 1459836505
    vehicle {
      id: "1_3646"
    }
  }
}


In [117]:
loc_timestamp = pos_header.find("timestamp")
loc_newline = pos_header.find("\n", loc_timestamp)
pos_timestamp=int(header[loc_timestamp:loc_newline].replace("timestamp: ",""))
print "Timestamp:\n",pos_timestamp

Timestamp:
1459836597


In [26]:
vehicles=[]
for i in pos_entities:
    loc_vehicle = i.find("vehicle")
    loc_cbrackets = [j.start() for j in re.finditer("}", i)]
    vehicles.append(i[loc_vehicle:loc_cbrackets[-2]+1])

print vehicles[0]

vehicle {
    position {
      latitude: 47.192806
      longitude: -122.2956
    }
    timestamp: 1459807691
    vehicle {
      id: "258"
    }
  }


In [27]:
trip_ids=[]
route_ids=[]
for i in vehicles:
    loc_trip = i.find("trip")
    if loc_trip == -1:
        trip_ids.append([])
        route_ids.append([])
        continue
    loc_trip_id = i.find("trip_id",loc_trip)
    loc_route_id = i.find("route_id", loc_trip)
    
    loc_newline = i.find("\n", loc_trip_id)
    trip_id = int(i[loc_trip_id:loc_newline].replace("trip_id: ","").replace('"',''))
    trip_ids.append(trip_id)
    
    loc_newline = i.find("\n", loc_route_id)
    route_id = int(i[loc_route_id:loc_newline].replace("route_id: ","").replace('"',''))
    route_ids.append(route_id)

print header
print trip_ids[0]
print route_ids[0]

header {
  gtfs_realtime_version: "1.0"
  timestamp: 1459832835
}
[]
[]


In [28]:
positions=[]
for i in vehicles:
    loc_position = i.find("position")
    if loc_position == -1:
        positions.append([])
        continue
    loc_cbracket = i.find("}",loc_position)
    positions.append(i[loc_position:loc_cbracket+1])

print positions[0]

position {
      latitude: 47.192806
      longitude: -122.2956
    }


In [29]:
latitudes=[]
for i in vehicles:
    loc_latitude = i.find("latitude")
    if loc_latitude == -1:
        latitudes.append([])
        continue
    loc_newline = i.find("\n",loc_latitude)
    num = float(i[loc_latitude:loc_newline].replace("latitude: ",""))
    latitudes.append(num)

print latitudes[0]

47.192806


In [30]:
longitudes=[]
for i in vehicles:
    loc_longitude = i.find("longitude")
    if loc_longitude == -1:
        longitudes.append([])
        continue
    loc_newline = i.find("\n",loc_longitude)
    num = float(i[loc_longitude:loc_newline].replace("longitude: ",""))
    longitudes.append(num)

print longitudes[0]

-122.2956


In [31]:
timestamps=[]
for i in vehicles:
    loc_timestamp = i.find("timestamp")
    if loc_longitude == -1:
        timestamps.append([])
        continue
    loc_newline = i.find("\n",loc_timestamp)
    num = int(i[loc_timestamp:loc_newline].replace("timestamp: ",""))
    timestamps.append(num)

print timestamps[0]

1459807691


In [33]:
vehicle_ids=[]
for i in vehicles:
    loc_vehicle_info = [j.start() for j in re.finditer("vehicle", i)]
    loc_vehicle_section = i.find("vehicle", loc_vehicle_info[1])
    if loc_vehicle_info == -1:
        vehicle_ids.append([])
        continue
    loc_vehicle_id = i.find("id", loc_vehicle_section)
    loc_newline = i.find("\n", loc_vehicle_id)
    num = int(i[loc_vehicle_id:loc_newline].replace("id: ","").replace('"',''))
    vehicle_ids.append(num)

print vehicle_ids[0]

258


In [56]:
with open('./Position_Data_Records/Positions_'+agency_ID+"_"+str(call_timestamp)+".csv", 'w') as myfile:
    wr = csv.writer(myfile, delimiter = ',', quoting=csv.QUOTE_NONE)
    wr.writerow(("Vehicle_Id", "Timestamp", "Trip_Id", "Route_Id", "Latitude", "Longitude"))
    for i in range(len(vehicles)):
        wr.writerow((vehicle_ids[i],timestamps[i],trip_ids[i],route_ids[i],latitudes[i],longitudes[i]))

### Get Current System Time

In [76]:
request_system_time = oba_url + 'api/where/current-time.xml?key=' + str(key_mpm)
request_system_time

'http://api.pugetsound.onebusaway.org/api/where/current-time.xml?key=66fae88a-fcc9-4ceb-958d-3932f378c161'

In [77]:
r = requests.get(request_system_time)
r.status_code

200

In [78]:
b = BS(r.content, "xml")
print b

<?xml version="1.0" encoding="utf-8"?>
<response>
<version>2</version>
<code>200</code>
<currentTime>1459837353223</currentTime>
<text>OK</text>
<data class="entryWithReferences">
<references/>
<entry class="time">
<time>1459837353223</time>
<readableTime>2016-04-04T23:22:33-07:00</readableTime>
</entry>
</data>
</response>


### Get Trip Updates Trial

In [134]:
request_trip_updates = oba_url + 'api/gtfs_realtime/trip-updates-for-agency/' + agency_ID + '.pbtext?key=' + key_mpm + "&removeAgencyIds=false"
request_trip_updates

'http://api.pugetsound.onebusaway.org/api/gtfs_realtime/trip-updates-for-agency/1.pbtext?key=66fae88a-fcc9-4ceb-958d-3932f378c161&removeAgencyIds=false'

In [135]:
r = requests.get(request_trip_updates)
r.status_code

200

In [136]:
trip_updates_raw = r.content
#print trip_updates_raw

In [137]:
begin_header = trip_updates_raw.find("header")
end_header = trip_updates_raw.find("}")
trip_updates_header = trip_updates_raw[begin_header:end_header+1]

loc_entity = [i.start() for i in re.finditer("entity", trip_updates_raw)]
trip_updates_entities = []
for i in loc_entity:
    if i!=loc_entity[-1]:
        trip_updates_entities.append(trip_updates_raw[i:(loc_entity[loc_entity.index(i)+1]-1)])
    else:
        trip_updates_entities.append(trip_updates_raw[i:-1])


In [138]:
print trip_updates_header
print trip_updates_entities[0]

header {
  gtfs_realtime_version: "1.0"
  timestamp: 1459838211
}
entity {
  id: "1"
  trip_update {
    trip {
      trip_id: "1_30973639"
      route_id: "1_100225"
    }
    stop_time_update {
      arrival {
      }
      departure {
        time: 1459839091
      }
      stop_id: "1_98360"
    }
    vehicle {
      id: "1_3646"
    }
    timestamp: 1459838125
    delay: 0
  }
}


In [140]:
loc_timestamp = trip_updates_header.find("timestamp")
loc_newline = trip_updates_header.find("\n", loc_timestamp)
trip_updates_timestamp=int(trip_updates_header[loc_timestamp:loc_newline].replace("timestamp: ",""))
print "Timestamp:\n",trip_updates_timestamp

Timestamp:
1459838211


In [141]:
vehicles=[]
for i in trip_updates_entities:
    loc_vehicle = i.find("vehicle")
    loc_cbrackets = [j.start() for j in re.finditer("}", i)]
    vehicles.append(i[loc_vehicle:loc_cbrackets[-2]+1])

print vehicles[0]

vehicle {
      id: "1_3646"
    }
    timestamp: 1459838125
    delay: 0
  }
